In [5]:
# read all the data from the csv files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
# read the data from the csvs files in the folder
data = []
for file in os.listdir("."):
    if file.endswith(".csv"):
        data.append(pd.read_csv(file).to_dict('records'))
        data[-1]["step"] = data[-1]["display_name"][5:]



TypeError: list indices must be integers or slices, not str

In [6]:
data

[[{'displayName': 'eval_step_0',
   'status': 'Completed',
   'createdOn': '2023-10-10T11:50:28.0940673+00:00',
   'duration': 60723.3949,
   'gpt_coherence': 4.14,
   'gpt_coherence_pass_rate(%)': 82.14,
   'gpt_fluency': 4.15,
   'gpt_fluency_pass_rate(%)': 75,
   'gpt_groundedness': 3.41,
   'gpt_groundedness_pass_rate(%)': 42.86,
   'gpt_relevance': 2.71,
   'gpt_relevance_pass_rate(%)': 39.29}]]